An example of creating a subclass of HyperImg and working with it

In [2]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import tifffile as tiff

import hyper_img as hyper

Pathes and constants

In [4]:
IMG_PATH = '/home/igor/projects/Seed/data/golozernost'
CALIBR_WHITE_PATH = IMG_PATH + '/2022_12_21_session_001_004_snapshot_cube.tiff'
CALIBR_BLACK_PATH = IMG_PATH + '/2022_12_21_session_001_005_snapshot_cube.tiff'
TARGET_VARIABLE = 'Голозерность/Пленчатость'

Example of HyperImg subclass

In [5]:
class GolozernostHyperImg(hyper.HyperImg):
    
    def _get_image(self) -> np.ndarray:
        img = tiff.imread(self.path)
        bl_img = tiff.imread(CALIBR_BLACK_PATH)
        new_img = np.where(bl_img > img, 0, img - bl_img)
        wh_img = tiff.imread(CALIBR_WHITE_PATH)
        try:
            return new_img /(wh_img - bl_img)
        except ZeroDivisionError:
            raise ZeroDivisionError('black image is equal to white in one component')

    
    def _get_target_varible(self) -> str:
        
        if 'голозерные' in self.path:
            return 'голозерные'
        
        if 'пленчатые' in self.path:
            return 'пленчатые'
        
        return 'калибровка'

Get a list of GolozernostHyperImg

In [6]:
img_lst: list[hyper.TableHyperImg] = hyper.get_list_hyper_img(path=IMG_PATH,
                                                             class_name=GolozernostHyperImg,
                                                             segmenter=hyper.PlainCv2Segmenter(25),
                                                             filter=lambda x: x != '',
                                                             target_varible_name=TARGET_VARIABLE)

In [7]:
hyper.get_count_group(img_lst)

{'калибровка': 2, 'пленчатые': 6, 'голозерные': 6}

Then we work with the list in the same way as in tableImg_example.ipynb (see my github)